<a href="https://colab.research.google.com/github/K-BRAHIM/meme_coin_scanner_colab/blob/main/%D9%85%D8%B7%D9%88%D8%B1_NASDAQ_100_%D8%AA%D8%AF%D8%A7%D9%88%D9%84_%D8%A8%D8%A7%D9%84_agent_22_08_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=a9423c149b56c51aabd23cc62946701fa6f5fe3c310ebeb0098fd47d6174dc1a
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
novandraanugrah_nasdaq_100_nas100_historical_price_data_path = kagglehub.dataset_download('novandraanugrah/nasdaq-100-nas100-historical-price-data')

print('Data source import complete.')


100%|██████████| 48.0M/48.0M [00:00<00:00, 110MB/s] 

Extracting files...


Data source import complete.


In [3]:
# =============================================================================
# الخلية 1: الإعداد، استيراد المكتبات، وتحميل البيانات
# =============================================================================
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import kagglehub
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# --- الخطوة أ: تحميل البيانات من Kaggle وإنشاء 'file_path' ---
print("Downloading dataset...")
file_path = kagglehub.dataset_download(
    'novandraanugrah/nasdaq-100-nas100-historical-price-data', # اسم الحزمة
    '15m_data.csv'                                           # اسم الملف
)
print(f"Download complete. Data is at: {file_path}")

# --- الخطوة ب: تعريف دالة تحميل البيانات ---
def load_and_prepare_data(path):
    """تحميل البيانات، تحويل الأعمدة، والفرز."""
    print(f"Loading data from {path}...")
    try:
        # تحديد الفاصل الصحيح عند القراءة
        df = pd.read_csv(path, sep='\t')
        if df.shape[1] == 1:
            df = pd.read_csv(path, sep=',')
    except Exception as e:
        raise ValueError(f"Error reading file: {e}")

    df.columns = [c.strip() for c in df.columns]
    df["DateTime"] = pd.to_datetime(df["DateTime"], utc=True)
    df = df.sort_values("DateTime").reset_index(drop=True)
    df["bar_pos"] = np.arange(len(df))
    print("✅ Data loaded and prepared successfully.")
    return df

# --- الخطوة ج: استدعاء الدالة باستخدام 'file_path' الصحيح ---
main_df = load_and_prepare_data(file_path)

# --- الخطوة د: عرض عينة من البيانات ---
print("\n--- 🔬 Sample of Initial Data ---")
display(main_df.head())

Download complete. Data is at: /kaggle/input/nasdaq-100-nas100-historical-price-data/15m_data.csv
Loading data from /kaggle/input/nasdaq-100-nas100-historical-price-data/15m_data.csv...
✅ Data loaded and prepared successfully.

--- 🔬 Sample of Initial Data ---


,DateTime,Open,High,Low,Close,Volume,TickVolume,bar_pos
0,2016-11-15 06:30:00+00:00,4699.6,4701.4,4699.6,4701.3,0,173,0
1,2016-11-15 06:45:00+00:00,4701.4,4703.1,4701.3,4701.6,0,276,1
2,2016-11-15 07:00:00+00:00,4701.7,4703.1,4700.7,4701.3,0,280,2
3,2016-11-15 07:15:00+00:00,4700.9,4702.9,4698.7,4701.3,0,450,3
4,2016-11-15 07:30:00+00:00,4701.3,4702.4,4700.4,4701.6,0,310,4


In [4]:
# =============================================================================
# Cell 2 (mُحدّثة v6): The Intuitive Naming System
# =============================================================================

def generate_weekly_lines(df):
    """
    Creates weekly structural lines with the new intuitive naming (e.g., 'W.1').
    """
    print("Generating weekly lines...")
    # ... (The logic for finding and filtering unique_week_starts remains the same) ...
    df['week_start_date'] = df['DateTime'].dt.to_period('W').apply(lambda r: r.start_time).dt.date
    unique_week_starts = sorted(df['week_start_date'].unique())
    if unique_week_starts:
        first_week_start_date = unique_week_starts[0]
        first_week_group = df[df['week_start_date'] == first_week_start_date]
        if first_week_group['DateTime'].iloc[0].date().weekday() != 0:
            print(f"⚠️ First week is incomplete. Discarding it.")
            unique_week_starts = unique_week_starts[1:]

    lines = []
    for i in tqdm(range(len(unique_week_starts) - 1), desc="Generating Weekly Lines", leave=False):
        current_week_date, next_week_date = unique_week_starts[i], unique_week_starts[i+1]
        current_group = df[df['week_start_date'] == current_week_date]
        next_group = df[df['week_start_date'] == next_week_date]
        if current_group.empty or next_group.empty: continue

        p0_row, p1_row = current_group.iloc[0], next_group.iloc[0]
        p0_pos, p0_price, p0_datetime = p0_row["bar_pos"], p0_row["Open"], p0_row["DateTime"]
        p1_pos, p1_price, p1_datetime = p1_row["bar_pos"], p1_row["Open"], p1_row["DateTime"]

        if (p1_pos - p0_pos) == 0: continue
        slope_m = (p1_price - p0_price) / (p1_pos - p0_pos)
        intercept_c = p0_price - slope_m * p0_pos

        lines.append({
            # *** NEW NAMING CONVENTION HERE ***
            "line_id": f"W.{i+1}",
            "line_type": "Weekly",
            "p0_datetime": p0_datetime, "p0_price": p0_price,
            "p1_datetime": p1_datetime, "p1_price": p1_price,
            "p0_pos": p0_pos, "p1_pos": p1_pos,
            "slope_m": slope_m, "intercept_c": intercept_c
        })

    lines_df = pd.DataFrame(lines)
    df.drop(columns=['week_start_date'], inplace=True, errors='ignore')
    print(f"✅ Created {len(lines_df)} weekly lines.")
    return lines_df

def generate_monthly_lines(df):
    """
    Creates monthly structural lines with the new intuitive naming (e.g., 'M.1').
    """
    print("Generating monthly lines...")
    # ... (The logic for finding and filtering unique_month_starts remains the same) ...
    df['month_start_date'] = df['DateTime'].dt.to_period('M').apply(lambda r: r.start_time).dt.date
    unique_month_starts = sorted(df['month_start_date'].unique())
    if unique_month_starts:
        first_month_start_date = unique_month_starts[0]
        first_month_group = df[df['month_start_date'] == first_month_start_date]
        if first_month_group['DateTime'].iloc[0].date().day != 1:
            print(f"⚠️ First month is incomplete. Discarding it.")
            unique_month_starts = unique_month_starts[1:]

    lines = []
    for i in tqdm(range(len(unique_month_starts) - 1), desc="Generating Monthly Lines", leave=False):
        current_month_date, next_month_date = unique_month_starts[i], unique_month_starts[i+1]
        current_group = df[df['month_start_date'] == current_month_date]
        next_group = df[df['month_start_date'] == next_month_date]
        if current_group.empty or next_group.empty: continue

        p0_row, p1_row = current_group.iloc[0], next_group.iloc[0]
        p0_pos, p0_price, p0_datetime = p0_row["bar_pos"], p0_row["Open"], p0_row["DateTime"]
        p1_pos, p1_price, p1_datetime = p1_row["bar_pos"], p1_row["Open"], p1_row["DateTime"]

        if (p1_pos - p0_pos) == 0: continue
        slope_m = (p1_price - p0_price) / (p1_pos - p0_pos)
        intercept_c = p0_price - slope_m * p0_pos

        lines.append({
            # *** NEW NAMING CONVENTION HERE ***
            "line_id": f"M.{i+1}",
            "line_type": "Monthly",
            "p0_datetime": p0_datetime, "p0_price": p0_price,
            "p1_datetime": p1_datetime, "p1_price": p1_price,
            "p0_pos": p0_pos, "p1_pos": p1_pos,
            "slope_m": slope_m, "intercept_c": intercept_c
        })

    lines_df = pd.DataFrame(lines)
    df.drop(columns=['month_start_date'], inplace=True, errors='ignore')
    print(f"✅ Created {len(lines_df)} monthly lines.")
    return lines_df

def generate_all_lines(df):
    """
    Generates and combines all lines with the new naming system.
    """
    print("--- 🧬 Starting Multi-Level Line Generation Engine ---")
    weekly_lines = generate_weekly_lines(df)
    monthly_lines = generate_monthly_lines(df)

    # We no longer need a separate numerical ID, the new string ID is our primary key.
    combined_lines_df = pd.concat([weekly_lines, monthly_lines], ignore_index=True)

    print(f"\n✅ All lines combined. Total count: {len(combined_lines_df)} lines.")
    return combined_lines_df

# --- Execution and Output ---
lines_df = generate_all_lines(main_df)
print("\n--- 📈 Combined Lines Data (with new intuitive IDs) ---")

print("\n--- Weekly Lines (Sample from the beginning) ---")
display(lines_df[['line_id', 'line_type', 'p0_datetime', 'p1_datetime']].head())

print("\n--- Monthly Lines (Sample from the end) ---")
display(lines_df[['line_id', 'line_type', 'p0_datetime', 'p1_datetime']].tail())

--- 🧬 Starting Multi-Level Line Generation Engine ---
Generating weekly lines...
⚠️ First week is incomplete. Discarding it.


Generating Weekly Lines:   0%|          | 0/452 [00:00<?, ?it/s]

✅ Created 452 weekly lines.
Generating monthly lines...
⚠️ First month is incomplete. Discarding it.


Generating Monthly Lines:   0%|          | 0/104 [00:00<?, ?it/s]

✅ Created 104 monthly lines.

✅ All lines combined. Total count: 556 lines.

--- 📈 Combined Lines Data (with new intuitive IDs) ---

--- Weekly Lines (Sample from the beginning) ---


,line_id,line_type,p0_datetime,p1_datetime
0,W.1,Weekly,2016-11-21 01:00:00+00:00,2016-11-28 01:00:00+00:00
1,W.2,Weekly,2016-11-28 01:00:00+00:00,2016-12-05 01:00:00+00:00
2,W.3,Weekly,2016-12-05 01:00:00+00:00,2016-12-12 01:00:00+00:00
3,W.4,Weekly,2016-12-12 01:00:00+00:00,2016-12-19 01:00:00+00:00
4,W.5,Weekly,2016-12-19 01:00:00+00:00,2016-12-27 02:00:00+00:00



--- Monthly Lines (Sample from the end) ---


,line_id,line_type,p0_datetime,p1_datetime
551,M.100,Monthly,2025-03-03 01:00:00+00:00,2025-04-01 01:00:00+00:00
552,M.101,Monthly,2025-04-01 01:00:00+00:00,2025-05-01 01:00:00+00:00
553,M.102,Monthly,2025-05-01 01:00:00+00:00,2025-06-02 01:00:00+00:00
554,M.103,Monthly,2025-06-02 01:00:00+00:00,2025-07-01 01:00:00+00:00
555,M.104,Monthly,2025-07-01 01:00:00+00:00,2025-08-01 01:00:00+00:00


In [5]:
# =============================================================================
# الخلية 3 (مُحدّثة v2.1): إضافة تاريخ إنشاء الخط
# =============================================================================

# =============================================================================
# Corrected Cell: The Interaction Analysis Engine
# =============================================================================

def find_and_analyze_interactions(df, lines_df):
    """
    Detects and analyzes each interaction between a candle and a line, extracting rich details.
    This version is compatible with the new string-based line_id system.
    """
    print("--- 🕵️‍♂️ Starting Interaction Analysis Engine ---")

    # --- Phase 1: Detecting Interactions ---
    print("Phase 1: Detecting interaction candles...")
    line_prices = np.zeros((len(df), len(lines_df)))
    bar_pos = df['bar_pos'].to_numpy()

    for i, line in tqdm(lines_df.iterrows(), total=len(lines_df), desc="Calculating Line Prices"):
        line_prices[:, i] = line['slope_m'] * bar_pos + line['intercept_c']

    # Use the numerical index of lines_df for columns, which is safer
    lines_prices_df = pd.DataFrame(line_prices, columns=lines_df.index)

    start_positions = lines_df['p0_pos'].to_numpy()
    time_mask = bar_pos[:, np.newaxis] >= start_positions

    interaction_matrix = (lines_prices_df.to_numpy() <= df['High'].to_numpy()[:, np.newaxis]) & \
                         (lines_prices_df.to_numpy() >= df['Low'].to_numpy()[:, np.newaxis]) & \
                         (time_mask)

    interaction_mask = interaction_matrix.any(axis=1)
    interaction_df = df[interaction_mask].copy()

    # Find the numerical index (position) of the line that was interacted with
    relevant_interactions = interaction_matrix[interaction_mask]
    interacted_line_indices = np.argmax(relevant_interactions, axis=1)

    # *** FIX: Directly look up the line information using the found indices ***
    # This is more efficient and avoids the merge error.
    interaction_df['interacted_line_id'] = lines_df.loc[interacted_line_indices, 'line_id'].values
    interaction_df['line_type'] = lines_df.loc[interacted_line_indices, 'line_type'].values
    interaction_df['line_creation_date'] = lines_df.loc[interacted_line_indices, 'p0_datetime'].values

    print(f"✅ Found {len(interaction_df)} total interaction candles.")

    # --- Phase 2: Analyzing Interaction Details ---
    print("\nPhase 2: Analyzing interaction details...")
    interaction_rows = np.where(interaction_mask)[0]
    interacted_line_prices = lines_prices_df.to_numpy()[interaction_rows, interacted_line_indices]
    interaction_df['line_price_at_touch'] = interacted_line_prices

    interaction_df['candle_direction'] = np.where(interaction_df['Close'] > interaction_df['Open'], 'Bullish', 'Bearish')

    body_min = interaction_df[['Open', 'Close']].min(axis=1)
    body_max = interaction_df[['Open', 'Close']].max(axis=1)

    interaction_df['interaction_type'] = np.where(
        (interaction_df['line_price_at_touch'] >= body_min) & (interaction_df['line_price_at_touch'] <= body_max),
        'Body Breakthrough',
        'Wick Touch'
    )

    print("✅ Detailed analysis complete.")
    return interaction_df

# --- Execution and Analytical Output ---
interaction_details_df = find_and_analyze_interactions(main_df, lines_df)

# Updated display columns
display_cols = [
    'DateTime', 'TickVolume', 'line_type', 'interacted_line_id',
    'line_creation_date', 'candle_direction', 'interaction_type'
]

print("\n--- 📊 Analytical Results of Interactions (Sample from beginning) ---")
display(interaction_details_df[display_cols].head())

print("\n--- 📊 Analytical Results of Interactions (Sample from end) ---")
display(interaction_details_df[display_cols].tail())

--- 🕵️‍♂️ Starting Interaction Analysis Engine ---
Phase 1: Detecting interaction candles...


Calculating Line Prices:   0%|          | 0/556 [00:00<?, ?it/s]

✅ Found 57472 total interaction candles.

Phase 2: Analyzing interaction details...
✅ Detailed analysis complete.

--- 📊 Analytical Results of Interactions (Sample from beginning) ---


,DateTime,TickVolume,line_type,interacted_line_id,line_creation_date,candle_direction,interaction_type
342,2016-11-21 01:00:00+00:00,323,Weekly,W.1,2016-11-21 01:00:00,Bullish,Body Breakthrough
346,2016-11-21 02:00:00+00:00,192,Weekly,W.1,2016-11-21 01:00:00,Bearish,Body Breakthrough
347,2016-11-21 02:15:00+00:00,160,Weekly,W.1,2016-11-21 01:00:00,Bullish,Body Breakthrough
382,2016-11-21 11:00:00+00:00,457,Weekly,W.1,2016-11-21 01:00:00,Bearish,Wick Touch
385,2016-11-21 11:45:00+00:00,262,Weekly,W.1,2016-11-21 01:00:00,Bullish,Wick Touch



--- 📊 Analytical Results of Interactions (Sample from end) ---


,DateTime,TickVolume,line_type,interacted_line_id,line_creation_date,candle_direction,interaction_type
202798,2025-08-01 05:30:00+00:00,992,Monthly,M.104,2025-07-01 01:00:00,Bullish,Wick Touch
202802,2025-08-01 06:30:00+00:00,930,Monthly,M.104,2025-07-01 01:00:00,Bearish,Body Breakthrough
202803,2025-08-01 06:45:00+00:00,1023,Monthly,M.104,2025-07-01 01:00:00,Bearish,Wick Touch
202842,2025-08-01 16:30:00+00:00,7068,Monthly,M.38,2020-01-02 01:00:00,Bearish,Body Breakthrough
202845,2025-08-01 17:15:00+00:00,6587,Monthly,M.38,2020-01-02 01:00:00,Bullish,Wick Touch


In [6]:
# =============================================================================
# الخلية 4 (مُحدّثة v2): خريطة المسار الديناميكية
# =============================================================================

# =============================================================================
# Corrected Cell: Dynamic Path Map (v2.1)
# =============================================================================

def create_dynamic_path_map(interaction_df):
    """
    Builds a path map by pairing consecutive transition events. This method is
    robust and avoids index lookup errors.
    """
    print("--- 🗺️ Building Dynamic Path Map (Corrected Logic) ---")

    # 1. Identify the first candle of each new interaction (the "arrival" events)
    transition_mask = interaction_df['interacted_line_id'] != interaction_df['interacted_line_id'].shift(1)
    arrival_df = interaction_df[transition_mask].copy()

    # 2. Build the path map by pairing each arrival event with the one before it
    # The 'departure' from line X is simply the 'arrival' at line X.
    path_map_df = pd.DataFrame({
        # Departure Info (from the PREVIOUS arrival event)
        'departure_time': arrival_df['DateTime'].shift(1),
        'departure_price': arrival_df['Close'].shift(1),
        'departure_type': arrival_df['interaction_type'].shift(1),
        'departure_bar_pos': arrival_df['bar_pos'].shift(1),
        'current_line_id': arrival_df['interacted_line_id'].shift(1),

        # Arrival Info (from the CURRENT arrival event)
        'arrival_time': arrival_df['DateTime'],
        'arrival_price': arrival_df['Close'],
        'arrival_type': arrival_df['interaction_type'],
        'arrival_bar_pos': arrival_df['bar_pos'],
        'next_line_id': arrival_df['interacted_line_id'],
    })

    # 3. Drop the first row which has no 'departure' info (it will have NaNs from .shift())
    path_map_df.dropna(inplace=True)

    # 4. Calculate contextual metrics
    path_map_df['time_in_bars'] = (path_map_df['arrival_bar_pos'] - path_map_df['departure_bar_pos']).astype(int)
    path_map_df['price_change'] = path_map_df['arrival_price'] - path_map_df['departure_price']
    path_map_df['target_price_based'] = (path_map_df['price_change'] > 0).astype(int)

    print(f"✅ Dynamic path map created. Found {len(path_map_df)} full transitions.")
    return path_map_df

# --- Execution and Output ---
path_map_df = create_dynamic_path_map(interaction_details_df)

# Display columns showing the dynamics of the movement
display_cols = [
    'current_line_id',
    'departure_type', # <-- How we left the current line
    'next_line_id',
    'arrival_type',   # <-- How we arrived at the next line
    'price_change',
    'time_in_bars',
    'target_price_based'
]

display(path_map_df[display_cols].head())

--- 🗺️ Building Dynamic Path Map (Corrected Logic) ---
✅ Dynamic path map created. Found 15358 full transitions.


,current_line_id,departure_type,next_line_id,arrival_type,price_change,time_in_bars,target_price_based
772,W.1,Body Breakthrough,W.2,Body Breakthrough,50.1,430,1
834,W.2,Body Breakthrough,W.1,Wick Touch,7.0,62,1
1041,W.1,Wick Touch,M.1,Body Breakthrough,-49.7,207,0
1107,M.1,Body Breakthrough,W.2,Body Breakthrough,-67.7,66,0
1226,W.2,Body Breakthrough,W.3,Wick Touch,-37.1,119,0


In [7]:
# =============================================================================
# الخلية 5 (جديدة): بناء الإطار الزمني الرئيسي (النسخة الكاملة)
# =============================================================================

def create_master_timeline(interaction_df, path_map_df):
    """
    يدمج سجل التفاعلات التفصيلي مع خريطة المسار الديناميكية
    لإنشاء إطار زمني موحد يحتوي على كل البيانات.
    """
    print("--- 🏗️ Building Master Analytical Timeline ---")

    # 1. تجهيز خريطة المسار للدمج
    # *** FIX: Added 'price_change' to the list of columns to be merged ***
    path_context_df = path_map_df[[
        'departure_time', 'current_line_id', 'next_line_id',
        'departure_type', 'arrival_type', 'target_price_based', 'price_change'
    ]].rename(columns={'departure_time': 'journey_start_time'})

    # FIX: Ensure the merge key has the correct UTC timezone
    path_context_df['journey_start_time'] = pd.to_datetime(path_context_df['journey_start_time'], utc=True)

    # 2. تنفيذ الدمج الزمني (merge_asof)
    interaction_df_sorted = interaction_df.sort_values('DateTime')
    path_context_df_sorted = path_context_df.sort_values('journey_start_time')

    master_df = pd.merge_asof(
        left=interaction_df_sorted,
        right=path_context_df_sorted,
        left_on='DateTime',
        right_on='journey_start_time',
        direction='backward'
    )

    # 3. تنظيف النتائج
    master_df.dropna(subset=['journey_start_time'], inplace=True)
    master_df = master_df[master_df['interacted_line_id'] == master_df['current_line_id']]

    print("✅ Master timeline created successfully.")
    return master_df

# --- التنفيذ وعرض المخرجات ---
master_timeline_df = create_master_timeline(interaction_details_df, path_map_df)

# *** FIX: Added 'TickVolume' and 'price_change' to the display list ***
display_cols = [
    'DateTime',
    'TickVolume',            # <-- Added
    'interacted_line_id',
    'interaction_type',
    'candle_direction',
    'current_line_id',
    'next_line_id',
    'departure_type',
    'price_change',          # <-- Added
    'target_price_based'
]

print("\n--- 📊 Master Timeline with All Data ---")
display(master_timeline_df[display_cols].head())

--- 🏗️ Building Master Analytical Timeline ---
✅ Master timeline created successfully.

--- 📊 Master Timeline with All Data ---


,DateTime,TickVolume,interacted_line_id,interaction_type,candle_direction,current_line_id,next_line_id,departure_type,price_change,target_price_based
0,2016-11-21 01:00:00+00:00,323,W.1,Body Breakthrough,Bullish,W.1,W.2,Body Breakthrough,50.1,1
1,2016-11-21 02:00:00+00:00,192,W.1,Body Breakthrough,Bearish,W.1,W.2,Body Breakthrough,50.1,1
2,2016-11-21 02:15:00+00:00,160,W.1,Body Breakthrough,Bullish,W.1,W.2,Body Breakthrough,50.1,1
3,2016-11-21 11:00:00+00:00,457,W.1,Wick Touch,Bearish,W.1,W.2,Body Breakthrough,50.1,1
4,2016-11-21 11:45:00+00:00,262,W.1,Wick Touch,Bullish,W.1,W.2,Body Breakthrough,50.1,1


In [8]:
# =============================================================================
# الخلية 6 (مُحدّثة): محرك تحليل شخصية الخطوط المتقدم
# =============================================================================

def create_advanced_line_profiles(interaction_df, path_map_df, main_df):
    """
    يبني ملفات تعريف شخصية متقدمة لكل خط، مدمجاً إحصائيات التفاعل،
    النتائج الاستراتيجية (من رحلة لأخرى)، والنتائج التكتيكية (رد الفعل الفوري).
    """
    print("--- 🧠 Starting Advanced Line Personality Analysis Engine ---")

    # --- 1. تحليل إحصائيات التفاعل (من interaction_df) ---
    print("Analyzing interaction statistics...")

    # المجموعة الأساسية للتحليل
    line_gb = interaction_df.groupby('interacted_line_id')

    # حساب المقاييس الأساسية
    touch_counts = line_gb.size().to_frame('total_touches')
    avg_volume = line_gb['TickVolume'].mean().to_frame('avg_volume_on_touch')

    # تفصيل أنواع التفاعلات
    interaction_type_counts = interaction_df.pivot_table(
        index='interacted_line_id', columns='interaction_type', aggfunc='size', fill_value=0
    ).add_suffix('_count')

    # تفصيل اختراقات الجسم حسب اتجاه الشمعة
    breakthrough_df = interaction_df[interaction_df['interaction_type'] == 'Body Breakthrough']
    breakthrough_direction_counts = breakthrough_df.pivot_table(
        index='interacted_line_id', columns='candle_direction', aggfunc='size', fill_value=0
    ).rename(columns={'Bullish': 'bullish_breakthrough_count', 'Bearish': 'bearish_breakthrough_count'})

    # --- 2. تحليل النتائج الاستراتيجية (من path_map_df) ---
    print("Analyzing strategic outcomes (journey-based)...")
    path_gb = path_map_df.groupby('current_line_id')

    # متوسط نتيجة الرحلة الكاملة
    avg_journey_price_change = path_gb['price_change'].mean().to_frame('avg_journey_price_change')

    # --- 3. تحليل النتائج التكتيكية (رد الفعل قصير المدى) ---
    print("Analyzing tactical outcomes (short-term reaction)...")
    REACTION_WINDOW = 12 # عدد الشموع (3 ساعات) لقياس رد الفعل

    # حساب التغير في السعر للـ 12 شمعة القادمة
    # نستخدم main_df للحصول على أسعار الإغلاق المستقبلية
    future_closes = main_df['Close'].shift(-REACTION_WINDOW)
    interaction_df['future_price'] = future_closes.loc[interaction_df.index]
    interaction_df['short_term_reaction'] = interaction_df['future_price'] - interaction_df['Close']

    # حساب متوسط رد الفعل لكل خط
    avg_short_term_reaction = interaction_df.groupby('interacted_line_id')['short_term_reaction'].mean().to_frame('avg_short_term_reaction')

    # --- 4. تجميع كل الإحصائيات في ملف شخصية واحد ---
    print("Combining all metrics into final profiles...")
    line_profiles_df = pd.concat([
        touch_counts,
        interaction_type_counts,
        breakthrough_direction_counts,
        avg_volume,
        avg_journey_price_change,
        avg_short_term_reaction
    ], axis=1).fillna(0) # تعبئة القيم المفقودة بصفر

    # حساب مقاييس إضافية (نسب مئوية) لسهولة القراءة
    line_profiles_df['breakthrough_rate_%'] = (line_profiles_df['Body Breakthrough_count'] / line_profiles_df['total_touches'] * 100).fillna(0)

    print("✅ Advanced line personality profiles created successfully.")
    return line_profiles_df

# --- التنفيذ وعرض المخرجات ---
# نستخدم interaction_details_df لأنه يحتوي على كل التفاصيل
line_profiles_df = create_advanced_line_profiles(interaction_details_df, path_map_df, main_df)

display(line_profiles_df.head())

--- 🧠 Starting Advanced Line Personality Analysis Engine ---
Analyzing interaction statistics...
Analyzing strategic outcomes (journey-based)...
Analyzing tactical outcomes (short-term reaction)...
Combining all metrics into final profiles...
✅ Advanced line personality profiles created successfully.


,total_touches,Body Breakthrough_count,Wick Touch_count,bearish_breakthrough_count,bullish_breakthrough_count,avg_volume_on_touch,avg_journey_price_change,avg_short_term_reaction,breakthrough_rate_%
M.1,128,63,65,29,34,2008.867188,-1.684211,-15.123438,49.218750
M.10,109,53,56,25,28,606.761468,3.295000,1.224771,48.623853
M.100,61,32,29,15,17,3853.114754,-1.270588,-13.245902,52.459016
M.101,55,27,28,13,14,4638.527273,70.175000,30.790909,49.090909
M.102,70,28,42,14,14,2859.871429,39.253333,24.515714,40.000000


In [9]:
# =============================================================================
# الخلية 7 (مُحدّثة v2): إضافة ترميز أحداث التفاعل
# =============================================================================

def encode_line_id(line_id_series):
    """
    يقوم بترميز عمود معرفات الخطوط النصية (مثل 'W.10') إلى عمودين رقميين.
    """
    line_id_str = line_id_series.astype(str)
    type_encoded = pd.Series(0, index=line_id_series.index)
    type_encoded[line_id_str.str.startswith('W')] = 1
    type_encoded[line_id_str.str.startswith('M')] = 2
    seq_encoded = line_id_str.str.split('.').str[1].fillna(0).astype(int)
    return type_encoded, seq_encoded

def create_hybrid_data(main_df, interaction_df, line_profiles_df):
    """
    يدمج كل مصادر البيانات ويجهزها في إطار بيانات شامل ومرمّز بالكامل للوكيل.
    """
    print("--- 🤖 Preparing Comprehensive Data for the Expert Agent ---")

    # 1. دمج معلومات التفاعل (الآن ندمج كل التفاصيل اللازمة للترميز)
    interaction_tags = interaction_df[['DateTime', 'interacted_line_id', 'interaction_type', 'candle_direction']]
    df = pd.merge(main_df.copy(), interaction_tags, on='DateTime', how='left')

    # 2. دمج "شخصية" الخطوط
    df = pd.merge(df, line_profiles_df, left_on='interacted_line_id', right_index=True, how='left')

    # 3. ملء القيم الفارغة
    df['interacted_line_id'].fillna("None", inplace=True)
    df[line_profiles_df.columns] = df[line_profiles_df.columns].fillna(0)

    # 4. ترميز معرف الخط
    print("Encoding line IDs...")
    df['line_type_encoded'], df['line_seq_encoded'] = encode_line_id(df['interacted_line_id'])

    # 5. ترميز أحداث التفاعل (الإضافة الجديدة والمهمة)
    print("Encoding interaction events...")
    df['interaction_event_encoded'] = 0 # القيمة الافتراضية (لا يوجد تفاعل)
    df.loc[df['interaction_type'] == 'Wick Touch', 'interaction_event_encoded'] = 1
    df.loc[(df['interaction_type'] == 'Body Breakthrough') & (df['candle_direction'] == 'Bullish'), 'interaction_event_encoded'] = 2
    df.loc[(df['interaction_type'] == 'Body Breakthrough') & (df['candle_direction'] == 'Bearish'), 'interaction_event_encoded'] = 3

    # 6. إزالة الأعمدة النصية غير اللازمة للوكيل
    df.drop(columns=['interacted_line_id', 'interaction_type', 'candle_direction'], inplace=True)

    print("✅ All data merged and fully encoded into a comprehensive DataFrame.")
    return df

# --- التنفيذ وعرض المخرجات ---
hybrid_data_df = create_hybrid_data(main_df, interaction_details_df, line_profiles_df)

# أعمدة العرض لتوضيح النتيجة النهائية
display_cols = [
    'DateTime',
    'Close',
    'line_type_encoded',
    'line_seq_encoded',
    'interaction_event_encoded', # <-- عمود الترميز الجديد
    'total_touches',
    'breakthrough_rate_%'
]

# عرض عينة من الشموع التي بها تفاعل للتأكد من الترميز
display(hybrid_data_df[hybrid_data_df['interaction_event_encoded'] != 0][display_cols].head())

--- 🤖 Preparing Comprehensive Data for the Expert Agent ---
Encoding line IDs...
Encoding interaction events...
✅ All data merged and fully encoded into a comprehensive DataFrame.


,DateTime,Close,line_type_encoded,line_seq_encoded,interaction_event_encoded,total_touches,breakthrough_rate_%
342,2016-11-21 01:00:00+00:00,4813.6,1,1,2,61.0,49.180328
346,2016-11-21 02:00:00+00:00,4806.5,1,1,3,61.0,49.180328
347,2016-11-21 02:15:00+00:00,4809.3,1,1,2,61.0,49.180328
382,2016-11-21 11:00:00+00:00,4812.3,1,1,1,61.0,49.180328
385,2016-11-21 11:45:00+00:00,4812.3,1,1,1,61.0,49.180328


In [10]:
# =============================================================================
# الخلية 8 (مُصحّحة): بيئة التداول المتقدمة
# =============================================================================
import gymnasium as gym
from gymnasium import spaces
from collections import deque
import numpy as np

# --- ثوابت البيئة ---
WINDOW_SIZE = 92
SEQUENCE_LENGTH = 5
INITIAL_BALANCE = 10000

class AdvancedTradingEnv(gym.Env):
    def __init__(self, df):
        super(AdvancedTradingEnv, self).__init__()
        self.df = df
        self.profile_cols = [
            'total_touches', 'Body Breakthrough_count', 'Wick Touch_count',
            'bullish_breakthrough_count', 'bearish_breakthrough_count',
            'avg_volume_on_touch', 'avg_journey_price_change',
            'avg_short_term_reaction', 'breakthrough_rate_%'
        ]
        self.action_space = spaces.Discrete(3)
        vector_feature_count = 4 + len(self.profile_cols)
        self.observation_space = spaces.Dict({
            "image": spaces.Box(low=0, high=1, shape=(WINDOW_SIZE, 4, 1), dtype=np.float32),
            "vector": spaces.Box(low=-np.inf, high=np.inf, shape=(vector_feature_count * SEQUENCE_LENGTH,), dtype=np.float32)
        })

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.balance = INITIAL_BALANCE
        self.net_worth = INITIAL_BALANCE
        # *** FIX: Initialize previous_net_worth here ***
        self.previous_net_worth = INITIAL_BALANCE
        self.current_step = WINDOW_SIZE + SEQUENCE_LENGTH
        self.position = 0
        self.entry_price = 0
        self.vector_history = deque(maxlen=SEQUENCE_LENGTH)
        for _ in range(SEQUENCE_LENGTH):
            self._update_vector_history()
        return self._get_observation(), {}

    def _update_vector_history(self):
        current_features = self.df.iloc[self.current_step]
        profile_values = current_features[self.profile_cols].values
        vector_obs = np.concatenate([[
                current_features['line_type_encoded'], current_features['line_seq_encoded'],
                current_features['interaction_event_encoded'], current_features['TickVolume']
            ], profile_values])
        self.vector_history.append(vector_obs)

    def _get_observation(self):
        frame_ohlc = self.df[['Open', 'High', 'Low', 'Close']].iloc[self.current_step - WINDOW_SIZE : self.current_step].values
        last_close = frame_ohlc[-1, 3]
        frame_range = np.max(frame_ohlc) - np.min(frame_ohlc) + 1e-9
        image_obs = np.expand_dims((frame_ohlc - last_close) / frame_range, axis=2)
        vector_obs_flat = np.array(self.vector_history).flatten()
        return {"image": image_obs.astype(np.float32), "vector": vector_obs_flat.astype(np.float32)}

    def step(self, action):
        current_price = self.df['Close'].iloc[self.current_step]
        reward = 0

        # --- Reward Shaping Logic ---
        current_interaction = self.df['interaction_event_encoded'].iloc[self.current_step] > 0
        if self.position == 0:
            if action == 1 and not current_interaction: reward = -10
            elif action == 1 and current_interaction: reward = 2
            elif action != 1 and current_interaction: reward = -2

        # --- Action Execution ---
        if action == 1 and self.position == 0 and current_interaction:
            self.position = 1
            self.entry_price = current_price
        elif action == 2 and self.position == 1:
            self.balance += (current_price - self.entry_price)
            self.position = 0

        # --- PnL Reward Calculation (Corrected Logic) ---
        self.net_worth = self.balance
        if self.position == 1:
            self.net_worth += (current_price - self.entry_price)

        # *** FIX: Calculate reward as the change from the PREVIOUS step's net worth ***
        reward += self.net_worth - self.previous_net_worth
        self.previous_net_worth = self.net_worth # Update for the next step

        # --- State Update ---
        self.current_step += 1
        self._update_vector_history()

        done = self.net_worth <= 0 or self.current_step >= len(self.df) - 1
        return self._get_observation(), reward, done, False, {}

print("✅ تم تعريف بيئة التداول المتقدمة (AdvancedTradingEnv) بالنسخة المصححة.")

✅ تم تعريف بيئة التداول المتقدمة (AdvancedTradingEnv) بالنسخة المصححة.


In [11]:
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.4 MB/s eta 0:00:00


In [12]:
!pip install stable-baselines3 gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 2.9 MB/s eta 0:00:00


In [ ]:
# =============================================================================
# الخلية 9 (مُحدّثة): محرك التدريب بتاريخ جديد
# =============================================================================
# استيراد المكتبات اللازمة للتدريب
!pip install -q stable-baselines3[extra] gymnasium
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback
import os

# --- 1. التقسيم الزمني الصارم للبيانات (بالتاريخ الجديد) ---
# *** تم التعديل هنا ***
EVAL_START_DATE = '2024-05-01'
train_df = hybrid_data_df[hybrid_data_df['DateTime'] < EVAL_START_DATE].reset_index(drop=True)
eval_df = hybrid_data_df[hybrid_data_df['DateTime'] >= EVAL_START_DATE].reset_index(drop=True)
print(f"بيانات التدريب: {len(train_df)} شمعة (قبل {EVAL_START_DATE})")
print(f"بيانات التقييم: {len(eval_df)} شمعة (بعد {EVAL_START_DATE})")
print("-" * 30)

# --- 2. إنشاء بيئات التدريب والتقييم ---
print("Creating training and evaluation environments...")
num_cpu = 8
train_env_fns = [lambda: AdvancedTradingEnv(train_df) for i in range(num_cpu)]
train_vec_env = SubprocVecEnv(train_env_fns)
train_vec_env = VecNormalize(train_vec_env, norm_obs=True, norm_reward=True, clip_reward=10.0)

eval_env_fn = lambda: AdvancedTradingEnv(eval_df)
eval_env = DummyVecEnv([eval_env_fn])
eval_env = VecNormalize(eval_env, training=False, norm_obs=True, norm_reward=False)

# --- 3. إعداد آلية الحفظ والتقييم الدوري ---
eval_callback = EvalCallback(eval_env, best_model_save_path='./best_model/',
                             log_path='./logs/', eval_freq=max(5000 // num_cpu, 1),
                             deterministic=True, render=False)

# --- 4. منطق استئناف التدريب أو بدء تدريب جديد ---
MODEL_PATH = "ppo_advanced_agent.zip"
STATS_PATH = "ppo_advanced_agent_stats.pkl"

if os.path.exists(MODEL_PATH):
    print("\nResuming training from a saved model...")
    professional_model = PPO.load(MODEL_PATH, env=train_vec_env)
    train_vec_env = VecNormalize.load(STATS_PATH, train_vec_env)
else:
    print("\nStarting new training...")
    professional_model = PPO('MultiInputPolicy', train_vec_env,
                            learning_rate=5e-5,
                            n_steps=2048,
                            batch_size=64,
                            verbose=1)

# --- 5. حلقة التدريب المرحلي ---
num_stages = 40
steps_per_stage = 10000
print(f"\nStarting staged training: {num_stages} stages of {steps_per_stage} steps each.")
print("=" * 50)

for stage in range(num_stages):
    print(f"\n--- 🏋️‍♂️ Starting Training Stage {stage + 1}/{num_stages} ---")
    professional_model.learn(total_timesteps=steps_per_stage,
                             callback=eval_callback,
                             reset_num_timesteps=False)

    professional_model.save(MODEL_PATH)
    train_vec_env.save(STATS_PATH)
    print(f"✅ Progress for stage {stage + 1} saved successfully!")
    print("-" * 50)

print("\n✅ Staged training complete.")

بيانات التدريب: 173358 شمعة (قبل 2024-05-01)
بيانات التقييم: 29488 شمعة (بعد 2024-05-01)
------------------------------
Creating training and evaluation environments...


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



Starting new training...
Using cpu device

Starting staged training: 40 stages of 10000 steps each.

--- 🏋️‍♂️ Starting Training Stage 1/40 ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
